<a href="https://colab.research.google.com/github/Luke-Williams-01/Luke-Williams-01/blob/main/Galactic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.optimize import curve_fit

In [16]:
# Constants
Mtotal = 1E11
Ttotal = 12E9
Sft = 1E9
TauSun = 10E9

# Constant star formation rate
SFR = Mtotal/Sft

# Time elapsed array
T = np.linspace(0, Ttotal, 12000)
Tl = len(T)

# Mass range array
M = np.linspace(0.5, 25, 100)
dM = M[1] - M[0]
Ml = len(M)

# Time of formation array
Tform = np.arange(0, Sft, 1E7)
dT = Tform[1] - Tform[0]
Tfl = len(Tform)


In [17]:
# Finding how the number of stars evolves over time

# Finding E0 = Mtotal/Mtot
def Mtot(M):
  return (M**(-1.35))

Int = Mtot(M)*dM
E0 = (SFR * dT)/np.sum(Int)
print(E0)

# Finding Number of stars per Mass interval
Stnumb = np.zeros(Ml)

for i in np.arange(0, Ml, 1):
  def Nstar(M):
    return (M[i]**(-2.35)) * E0 * dM
  Stnumb[i] = Nstar(M)

print(Stnumb)
print(M)

'''
N=[]
j=0
for i in range(0,len(Stbrn)):
    j+=Stbrn[i]
    N.append(j)
     
print(N)

plt.plot(np.log(Tform), np.log(N))
'''

326088255.5343859
[4.11421077e+08 1.59924210e+08 8.16645574e+07 4.84542352e+07
 3.16189546e+07 2.20351953e+07 1.61141258e+07 1.22260709e+07
 9.54965622e+06 7.63666797e+06 6.22670515e+06 5.16060536e+06
 4.33691502e+06 3.68864093e+06 3.17019189e+06 2.74972350e+06
 2.40448234e+06 2.11788244e+06 1.87761730e+06 1.67440984e+06
 1.50116645e+06 1.35239283e+06 1.22378340e+06 1.11192793e+06
 1.01409890e+06 9.28095099e+05 8.52125343e+05 7.84720852e+05
 7.24668633e+05 6.70960282e+05 6.22752281e+05 5.79334949e+05
 5.40107969e+05 5.04560941e+05 4.72257840e+05 4.42824489e+05
 4.15938418e+05 3.91320597e+05 3.68728663e+05 3.47951337e+05
 3.28803801e+05 3.11123860e+05 2.94768725e+05 2.79612318e+05
 2.65543002e+05 2.52461657e+05 2.40280051e+05 2.28919446e+05
 2.18309413e+05 2.08386809e+05 1.99094898e+05 1.90382598e+05
 1.82203822e+05 1.74516907e+05 1.67284123e+05 1.60471237e+05
 1.54047134e+05 1.47983490e+05 1.42254475e+05 1.36836498e+05
 1.31707983e+05 1.26849164e+05 1.22241910e+05 1.17869565e+05
 1.137

'\nN=[]\nj=0\nfor i in range(0,len(Stbrn)):\n    j+=Stbrn[i]\n    N.append(j)\n     \nprint(N)\n\nplt.plot(np.log(Tform), np.log(N))\n'

In [4]:
# Finding the Luminosity evolution of the stars


In [5]:
# V magnitude and colour graphs

# Creating arrays for the V magnitude/BV colour data
MagMass = np.array([0.8, 1, 1.25, 2, 3, 5, 9, 15, 25])
Vmag = np.array([6.5, 4.93, 4.2, 1, -0.5, -1.4, -3.7, -4.3, -5.2])
BVmag = np.array([1.02, 0.69, 0.59, 0.1, -0.12, -0.18, -0.3, -0.32, -0.32])


# 